In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PATH            = './data/'
data            = pd.read_csv("./data/driving_log.csv")

images_center   = []

for f in data.loc[:, 'center']:
    images_center.append(plt.imread(PATH + f))

images_center   = np.array(images_center)

steering        = data.loc[:, 'steering'].to_numpy()

In [40]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Flatten(input_shape = (160, 320, 3)))
model.add(Dense(1))

model.compile(loss = 'mse', optimizer = 'adam')
model.fit(X_train, y_train, validation_split = 0.2, shuffle = True, nb_epoch = 7) 

model.save('model.h5')


Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6428 samples, validate on 1608 samples
Epoch 1/7
6428/6428 [==============================] - 5s 797us/step - loss: 7614190.2253 - val_loss: 4634.2904
Epoch 2/7
6428/6428 [==============================] - 5s 764us/step - loss: 4191.4109 - val_loss: 9177.7172
Epoch 3/7
6428/6428 [==============================] - 5s 779us/step - loss: 2781.7390 - val_loss: 1925.3964
Epoch 4/7
5696/6428 [=========================>....] - ETA: 0s - loss: 2506.5604

KeyboardInterrupt: 